In [1]:
import pandas as pd
import numpy as np

In [16]:
file_path = "Aggregated_data_for_ML.xlsx"
df = pd.read_excel(file_path, skiprows= 2)
df

,Run_ID,Avg_temp,Avg_pH,Min_DO %,Inoculum size (mL),Max_RPM,Total feed of Lactose (mL),Total culture duration (days),Contamination (Y/N),Total media volume (mL),Max FPU/ml,Notes
0,TRICH_01,28.50,5.103,0.5,50,300,0,5,Y,3000,0.1,Low yeild
1,TRICH_02,29.30,4.950,0.6,50,250,0,5,Y,3000,0.3,Low yeild
2,TRICH_03,25.00,4.477,6.4,100,250,1400,6,N,4400,1.5,Medium Yeild
3,TRICH_04,25.60,4.680,12.8,100,250,1000,6,N,4000,2.0,Medium Yeild
4,TRICH_05,26.10,4.590,45.0,100,200,1000,6,N,4000,5.6,High Yeild
5,TRICH_06,25.11,4.501,50.0,100,250,1000,6,N,4000,4.8,Medium Yeild
6,TRICH_07,25.12,4.500,40.0,100,250,500,6,N,3500,4.2,Medium Yeild
7,TRICH_08,30.01,5.010,35.0,100,250,1000,6,N,4000,4.0,Medium Yeild
8,TRICH_09,25.01,4.503,40.0,100,250,1500,6,N,4500,5.8,High Yeild
9,TRICH_10,26.30,4.430,44.0,100,255,1500,4,N,4500,3.6,Medium Yeild


In [3]:
df = df.dropna(how= 'all')

In [4]:
target_count = 1000
original_count = df.shape[0]
new_samples = target_count - original_count

In [6]:
contam_dist = df['Contamination (Y/N)'].value_counts(normalize=True)
notes_dist = df['Notes'].value_counts(normalize=True)

In [7]:
numeric_cols = df.select_dtypes(include=[np.number]).columns

In [8]:
synthetic_data = pd.DataFrame()

for col in numeric_cols:
    col_data = df[col].dropna()
    synthetic_data[col] = np.random.normal(
        loc=col_data.mean(),
        scale=col_data.std(),
        size=new_samples
    ).round(3)

In [9]:
synthetic_data['Run_ID'] = [f'TRICH_{i:03}' for i in range(original_count+1, target_count+1)]

In [10]:
synthetic_data['Contamination (Y/N)'] = np.random.choice(
    contam_dist.index, size=new_samples, p=contam_dist.values
)
synthetic_data['Notes'] = np.random.choice(
    notes_dist.index, size=new_samples, p=notes_dist.values
)

In [11]:
cols_order = ['Run_ID'] + list(numeric_cols) + ['Contamination (Y/N)', 'Notes']
synthetic_data = synthetic_data[cols_order]

In [12]:
df_combined = pd.concat([df, synthetic_data], ignore_index=True)

In [13]:
output_path = "Aggregated_data_with_1000_runs.xlsx"
df_combined.to_excel(output_path, index=False)

print(f"✅ Augmented dataset saved to: {output_path}")

✅ Augmented dataset saved to: Aggregated_data_with_1000_runs.xlsx


In [14]:
new_agg_data = pd.read_excel("Aggregated_data_with_1000_runs.xlsx")
new_agg_data

,Run_ID,Avg_temp,Avg_pH,Min_DO %,Inoculum size (mL),Max_RPM,Total feed of Lactose (mL),Total culture duration (days),Contamination (Y/N),Total media volume (mL),Max FPU/ml,Notes
0,TRICH_01,28.500,5.103,0.500,50.000,300.000,0.000,5.000,Y,3000.000,0.100,Low yeild
1,TRICH_02,29.300,4.950,0.600,50.000,250.000,0.000,5.000,Y,3000.000,0.300,Low yeild
2,TRICH_03,25.000,4.477,6.400,100.000,250.000,1400.000,6.000,N,4400.000,1.500,Medium Yeild
3,TRICH_04,25.600,4.680,12.800,100.000,250.000,1000.000,6.000,N,4000.000,2.000,Medium Yeild
4,TRICH_05,26.100,4.590,45.000,100.000,200.000,1000.000,6.000,N,4000.000,5.600,High Yeild
...,...,...,...,...,...,...,...,...,...,...,...,...
995,TRICH_996,26.046,5.368,15.094,99.542,274.873,1766.908,6.019,N,4427.538,3.064,Low yeild
996,TRICH_997,27.531,5.010,9.457,78.238,278.679,2347.377,4.731,N,4181.267,5.264,Low yeild
997,TRICH_998,25.392,4.838,-5.681,112.245,275.986,1023.041,5.574,Y,3978.739,-4.843,Medium Yeild
998,TRICH_999,24.612,4.974,0.696,79.743,240.534,2326.233,5.326,N,3294.996,3.890,Medium Yeild
